In [3]:
import mlflow

In [4]:
#conexion hacia el server de mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [5]:
import pandas as pd
data=pd.read_csv('../Data/train.csv')

In [6]:
# Creacion del cliente
client = mlflow.tracking.MlflowClient()

In [7]:
experiments=client.search_experiments()

In [8]:
experiments

[<Experiment: artifact_location='file:///d:/EntregableMLA/notebooks/mlruns/142787257619671728', creation_time=1729558387091, experiment_id='142787257619671728', last_update_time=1729558387091, lifecycle_stage='active', name='CrediCorp', tags={}>,
 <Experiment: artifact_location='file:///d:/EntregableMLA/notebooks/mlruns/0', creation_time=1729558387077, experiment_id='0', last_update_time=1729558387077, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
filtered_experiments = list(filter(lambda k: k.name == 'CrediCorp', experiments))
if filtered_experiments:
    experiment = filtered_experiments[0]
else:
    print("No se encontró ningún experimento con el nombre 'CrediCorp'.")


In [10]:
experiment=list(filter(lambda k:k.name=='CrediCorp',experiments))[0]

In [11]:
experiment_id =experiment.experiment_id

In [12]:
experiment_id

'142787257619671728'

In [13]:
runs=client.search_runs(experiment_ids=experiment_id,
                        filter_string="",
                        run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
                        max_results=3,
                        order_by=["metrics.R2 DESC"])# ordenar por R2 desc porque va de 0 a 1

In [14]:
runs

[<Run: data=<RunData: metrics={'AUC': 0.8776,
  'Accuracy': 0.8611,
  'F1': 0.6204,
  'Kappa': 0.538,
  'MCC': 0.5481,
  'Prec': 0.7357,
  'Recall': 0.5365,
  'TT': 2.19}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'class_weight': 'None',
  'criterion': 'gini',
  'max_depth': 'None',
  'max_features': 'sqrt',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '1',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'monotonic_cst': 'None',
  'n_estimators': '100',
  'n_jobs': '-1',
  'oob_score': 'False',
  'random_state': '123',
  'verbose': '0',
  'warm_start': 'False'}, tags={'Run ID': '2403555beb644085a70149786bae9a82',
  'Run Time': '22.54',
  'Source': 'create_model',
  'URI': '66a2f81c',
  'USI': '4dcc',
  'mlflow.log-model.history': '[{"run_id": "2403555beb644085a70149786bae9a82", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2024

In [15]:
runs=runs[0]

In [16]:
runs

<Run: data=<RunData: metrics={'AUC': 0.8776,
 'Accuracy': 0.8611,
 'F1': 0.6204,
 'Kappa': 0.538,
 'MCC': 0.5481,
 'Prec': 0.7357,
 'Recall': 0.5365,
 'TT': 2.19}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'class_weight': 'None',
 'criterion': 'gini',
 'max_depth': 'None',
 'max_features': 'sqrt',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '100',
 'n_jobs': '-1',
 'oob_score': 'False',
 'random_state': '123',
 'verbose': '0',
 'warm_start': 'False'}, tags={'Run ID': '2403555beb644085a70149786bae9a82',
 'Run Time': '22.54',
 'Source': 'create_model',
 'URI': '66a2f81c',
 'USI': '4dcc',
 'mlflow.log-model.history': '[{"run_id": "2403555beb644085a70149786bae9a82", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-10-22 16:52:42.819783", "model_u

In [17]:
run_id=runs.data.tags['Run ID']

In [18]:
#como tenemos el id  podemos utilizarlo para obtener los artefactos
run_id

'2403555beb644085a70149786bae9a82'

In [33]:
import mlflow
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


# Predict on a Pandas DataFrame.
import pandas as pd
exclude_variables=['id','Surname']
loaded_model.predict(data.drop(columns=exclude_variables+['Exited']))

array([0, 0, 0, ..., 0, 0, 1], dtype=int8)

In [20]:
#utilizar en el entorno de produccion
#registrar un modelo el mejor pase por el proceso de validacion
# el modelo se registra en el servidor de mlflow
# se registra en funcion de run_id y poder colocar un nombre

mlflow.register_model(f"runs:/{run_id}/model", "Credito_model3")

Successfully registered model 'Credito_model3'.
2024/10/22 12:53:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Credito_model3, version 1
Created version '1' of model 'Credito_model3'.


<ModelVersion: aliases=[], creation_timestamp=1729619599603, current_stage='None', description='', last_updated_timestamp=1729619599603, name='Credito_model3', run_id='2403555beb644085a70149786bae9a82', run_link='', source='file:///d:/EntregableMLA/notebooks/mlruns/142787257619671728/2403555beb644085a70149786bae9a82/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [36]:
client.transition_model_version_stage(name='Credito_Prod',version=1,stage='Production')

<ModelVersion: aliases=[], creation_timestamp=1729620354002, current_stage='Production', description='', last_updated_timestamp=1729620669340, name='Credito_Prod', run_id='2403555beb644085a70149786bae9a82', run_link='', source='file:///d:/EntregableMLA/notebooks/mlruns/142787257619671728/2403555beb644085a70149786bae9a82/artifacts/model', status='READY', status_message='', tags={'Autor': 'Manuel Limachi'}, user_id='', version='1'>

In [37]:
#la carga del modelo en produccion es desde el estado de produccion
import mlflow
logged_model = f'models:/Credito_Prod/Production'


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
exclude_variables=['id','Surname']
loaded_model.predict(data.drop(columns=exclude_variables+['Exited']))

array([0, 0, 0, ..., 0, 0, 1], dtype=int8)

In [34]:
#tag al modelo:

registered_model = mlflow.register_model(logged_model,'Credito_Prod')

# Obtener la versión del modelo registrado

model_version = registered_model.version

# Asignar la etiqueta (tag) al modelo registrado

client.set_model_version_tag(

    name='Credito_Prod',

    version=model_version,

    key="Autor",

    value="Manuel Limachi"

)



Registered model 'Credito_Prod' already exists. Creating a new version of this model...
2024/10/22 13:05:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Credito_Prod, version 1
Created version '1' of model 'Credito_Prod'.


In [61]:
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    return JSONResponse(
        status_code=500,
        content={"message": "Ocurrió un error en el servidor.", "detail": str(exc)},
    )


In [62]:
import requests

try:
    response = requests.post("http://127.0.0.1:5000/api/2.0/mlflow/model-versions/create", json=data)
    response.raise_for_status()  # Lanza un error si la respuesta es un error HTTP
except requests.exceptions.HTTPError as http_err:
    print(f"Error HTTP: {http_err}")
except Exception as err:
    print(f"Error: {err}")


Error: Object of type DataFrame is not JSON serializable
